In [2]:
import json
import networkx as nx
import os

# Load JSON data
file_path = r'./../json_files/filtered_summary.json'
with open(file_path, 'r') as file:
    data = json.load(file)

arcs_file_path = r'./../json_files/character_to_arcs.json'
with open(arcs_file_path, 'r') as file:
    arcs_data = json.load(file)

# Define the chronological order of arcs
sorted_arcs = [
    "Romance Dawn Arc", "Orange Town Arc", "Syrup Village Arc",
    "Baratie Arc", "Arlong Park Arc", "Loguetown Arc", "Reverse Mountain Arc",
    "Whisky Peak Arc", "Little Garden Arc", "Drum Island Arc", "Arabasta Arc",
    "Jaya Arc", "Skypiea Arc", "Long Ring Long Land Arc", "Water 7 Arc",
    "Enies Lobby Arc", "Post-Enies Lobby Arc", "Thriller Bark Arc",
    "Sabaody Archipelago Arc", "Amazon Lily Arc", "Impel Down Arc",
    "Marineford Arc", "Post-War Arc", "Return to Sabaody Arc",
    "Fish-Man Island Arc", "Punk Hazard Arc", "Dressrosa Arc", "Zou Arc",
    "Whole Cake Island Arc", "Levely Arc", "Wano Country Arc", "Egghead Arc"
]

# Logical segmentation of arcs based on story progression
arc_segments = {
    "East Blue Saga": [
        "Romance Dawn Arc", "Orange Town Arc", "Syrup Village Arc",
        "Baratie Arc", "Arlong Park Arc", "Loguetown Arc"
    ],
    "Alabasta Saga": [
        "Reverse Mountain Arc", "Whisky Peak Arc", "Little Garden Arc",
        "Drum Island Arc", "Arabasta Arc"
    ],
    "Sky Island Saga": [
        "Jaya Arc", "Skypiea Arc"
    ],
    "Water 7 Saga": [
        "Long Ring Long Land Arc", "Water 7 Arc", "Enies Lobby Arc",
        "Post-Enies Lobby Arc"
    ],
    "Thriller Bark Saga": [
        "Thriller Bark Arc"
    ],
    "Summit War Saga": [
        "Sabaody Archipelago Arc", "Amazon Lily Arc", "Impel Down Arc",
        "Marineford Arc", "Post-War Arc"
    ],
    "Fish-Man Island Saga": [
        "Return to Sabaody Arc", "Fish-Man Island Arc"
    ],
    "Dressrosa Saga": [
        "Punk Hazard Arc", "Dressrosa Arc"
    ],
    "Yonko Saga": [
        "Zou Arc", "Whole Cake Island Arc", "Levely Arc", "Wano Country Arc",
        "Egghead Arc"
    ]
}

# Sort character arcs based on the chronological order
def sort_arcs_for_characters(arcs_data, sorted_arcs):
    arc_index = {arc: idx for idx, arc in enumerate(sorted_arcs)}
    sorted_character_arcs = {}
    for character, arcs in arcs_data.items():
        sorted_arcs_for_character = sorted(arcs, key=lambda arc: arc_index.get(arc, float('inf')))
        sorted_character_arcs[character] = sorted_arcs_for_character
    return sorted_character_arcs

sorted_character_arcs = sort_arcs_for_characters(arcs_data, sorted_arcs)

# Determine the first appearance arc for each character
first_appearance = {}
for character, arcs in sorted_character_arcs.items():
    if arcs:  # If the character has arcs listed
        first_appearance[character] = arcs[0]  # The first arc in their sorted list

first_appearance    

{'A O': 'Marineford Arc',
 'Monkey D. Luffy': 'Romance Dawn Arc',
 'Portgas D. Ace': 'Drum Island Arc',
 'Shanks': 'Romance Dawn Arc',
 'Squard': 'Marineford Arc',
 'Absalom': 'Thriller Bark Arc',
 'Donquixote Doflamingo': 'Jaya Arc',
 'Hogback': 'Thriller Bark Arc',
 'Agsilly': 'Marineford Arc',
 'Amadob': 'Marineford Arc',
 'Islewan': 'Marineford Arc',
 'Sengoku': 'Jaya Arc',
 'Andre': 'Marineford Arc',
 'Crocodile': 'Whisky Peak Arc',
 'Ninth': 'Romance Dawn Arc',
 'Rush': 'Whisky Peak Arc',
 'Arthur': 'Marineford Arc',
 'Doma': 'Marineford Arc',
 'Hangan': 'Marineford Arc',
 'Reforte': 'Marineford Arc',
 'Whitey Bay': 'Marineford Arc',
 'Atmos': 'Marineford Arc',
 'Haruta': 'Marineford Arc',
 'Jinbe': 'Arlong Park Arc',
 'Marco': 'Jaya Arc',
 'Vista': 'Marineford Arc',
 'Avalo Pizarro': 'Impel Down Arc',
 'Catarina Devon': 'Impel Down Arc',
 'Sanjuan Wolf': 'Impel Down Arc',
 'Shiryu': 'Impel Down Arc',
 'Vasco Shot': 'Impel Down Arc',
 'Baggaley': 'Marineford Arc',
 'Wallem': 'Mar

In [3]:
# Cumulative graph
G_cumulative = nx.Graph()

# Ensure the output directory exists
def ensure_directory_exists(directory_path):
    os.makedirs(directory_path, exist_ok=True)

relative_path = r'./../networks/one_piece_cumulative'
ensure_directory_exists(relative_path)

# Generate cumulative graphs
for saga_name, arcs in arc_segments.items():
    # Create a graph for the current saga
    G_segment = nx.Graph()
    
    # Filter characters whose first appearance is in the current saga
    for person, details in data.items():
        if first_appearance.get(person) in arcs:  # Check first appearance
            word_count = details.get("word_count", 0)
            mutual_links = details.get("mention_intersection", [])
            
            # Add the character as a node
            G_segment.add_node(person, word_count=word_count)
            
            # Add edges only between characters already in G_segment
            for mentioned in mutual_links:
                if mentioned in G_segment.nodes:
                    G_segment.add_edge(person, mentioned)

    # Add current saga's nodes and edges to the cumulative graph
    G_cumulative.add_nodes_from(G_segment.nodes(data=True))
    G_cumulative.add_edges_from(G_segment.edges(data=True))

    # Call this before saving each file
    relative_path = r'../networks/one_piece_cumulative'

    # Save the graph as a .gexf file
    filename = f"{saga_name}.gexf"
    output_path = os.path.join(relative_path, filename)
    nx.write_gexf(G_cumulative, output_path)
    print(f"Graph saved: {output_path}")


Graph saved: ../networks/one_piece_cumulative\East Blue Saga.gexf
Graph saved: ../networks/one_piece_cumulative\Alabasta Saga.gexf
Graph saved: ../networks/one_piece_cumulative\Sky Island Saga.gexf
Graph saved: ../networks/one_piece_cumulative\Water 7 Saga.gexf
Graph saved: ../networks/one_piece_cumulative\Thriller Bark Saga.gexf
Graph saved: ../networks/one_piece_cumulative\Summit War Saga.gexf
Graph saved: ../networks/one_piece_cumulative\Fish-Man Island Saga.gexf
Graph saved: ../networks/one_piece_cumulative\Dressrosa Saga.gexf
Graph saved: ../networks/one_piece_cumulative\Yonko Saga.gexf


**Old One working**

In [4]:
import math
import os
import json
import networkx as nx


# Cargar datos del archivo JSON de resumen
file_path = r'./../json_files/filtered_summary.json'
with open(file_path, 'r') as file:
    data = json.load(file)

# Cargar datos de los arcos desde el archivo JSON
arcs_file_path = r'./../json_files/character_to_arcs.json'
with open(arcs_file_path, 'r') as file:
    arcs_data = json.load(file)

# Lista de arcos ordenados cronológicamente
sorted_arcs = [
    "Romance Dawn Arc", "Orange Town Arc", "Syrup Village Arc", "Baratie Arc", "Arlong Park Arc", "Loguetown Arc",
    "Reverse Mountain Arc", "Whisky Peak Arc", "Little Garden Arc", "Drum Island Arc", "Arabasta Arc", "Jaya Arc",
    "Skypiea Arc", "Long Ring Long Land Arc", "Water 7 Arc", "Enies Lobby Arc", "Post-Enies Lobby Arc",
    "Thriller Bark Arc", "Sabaody Archipelago Arc", "Amazon Lily Arc", "Impel Down Arc", "Marineford Arc",
    "Post-War Arc", "Return to Sabaody Arc", "Fish-Man Island Arc", "Punk Hazard Arc", "Dressrosa Arc", "Zou Arc",
    "Whole Cake Island Arc", "Levely Arc", "Wano Country Arc", "Egghead Arc"
]

# Dividir los arcos en 5 segmentos
num_steps = 5
segment_size = len(sorted_arcs) // num_steps
remainder = len(sorted_arcs) % num_steps

# Crear los segmentos, asegurándose de que todos los arcos sean cubiertos
arc_segments = []
start = 0
for i in range(num_steps):
    # Asignar el tamaño base más el "extra" si es el segmento con el residuo
    end = start + segment_size + (1 if i < remainder else 0)
    arc_segments.append(sorted_arcs[start:end])
    start = end

# Generar una red para cada segmento de arcos
for i, segment in enumerate(arc_segments):
    # Crear un nuevo grafo
    G_segment = nx.Graph()

    # Iterar sobre los datos para añadir nodos y relaciones basados en los arcos del segmento
    for person, details in data.items():
        word_count = details.get("word_count", 0)
        mutual_links = details.get("mention_intersection", [])
        character_arcs = arcs_data.get(person, [])
        
        # Verificar si el personaje pertenece a alguno de los arcos en el segmento actual
        if any(arc in character_arcs for arc in segment):
            # Añadir el nodo al grafo con su atributo 'word_count'
            G_segment.add_node(person, word_count=word_count)

            # Añadir relaciones con menciones mutuas
            for mentioned in mutual_links:
                if mentioned in data:  # Asegurarse de que la mención exista en los datos
                    G_segment.add_edge(person, mentioned)

    # Guardar el grafo como archivo .gexf
    relative_path = r'./../networks/one_piece_cumulative'
    filename = f"cumulative_step_{i + 1}.gexf"
    nx.write_gexf(G_segment, os.path.join(relative_path,filename))
    print(f"Archivo guardado: {filename}")

Archivo guardado: cumulative_step_1.gexf
Archivo guardado: cumulative_step_2.gexf
Archivo guardado: cumulative_step_3.gexf
Archivo guardado: cumulative_step_4.gexf
Archivo guardado: cumulative_step_5.gexf


In [5]:
import math
import os
import json
import networkx as nx

# Define logical segmentation of arcs based on story progression
arc_segments = {
    "East Blue Saga": [
        "Romance Dawn Arc", "Orange Town Arc", "Syrup Village Arc",
        "Baratie Arc", "Arlong Park Arc", "Loguetown Arc"
    ],
    "Alabasta Saga": [
        "Reverse Mountain Arc", "Whisky Peak Arc", "Little Garden Arc",
        "Drum Island Arc", "Arabasta Arc"
    ],
    "Sky Island Saga": [
        "Jaya Arc", "Skypiea Arc"
    ],
    "Water 7 Saga": [
        "Long Ring Long Land Arc", "Water 7 Arc", "Enies Lobby Arc",
        "Post-Enies Lobby Arc"
    ],
    "Thriller Bark Saga": [
        "Thriller Bark Arc"
    ],
    "Summit War Saga": [
        "Sabaody Archipelago Arc", "Amazon Lily Arc", "Impel Down Arc",
        "Marineford Arc", "Post-War Arc"
    ],
    "Fish-Man Island Saga": [
        "Return to Sabaody Arc", "Fish-Man Island Arc"
    ],
    "Dressrosa Saga": [
        "Punk Hazard Arc", "Dressrosa Arc"
    ],
    "Yonko Saga": [
        "Zou Arc", "Whole Cake Island Arc", "Levely Arc", "Wano Country Arc",
        "Egghead Arc"
    ]
}

# Load data from JSON summary file
file_path = r'./../json_files/filtered_summary.json'
with open(file_path, 'r') as file:
    data = json.load(file)

# Load data of arcs from JSON file
arcs_file_path = r'./../json_files/character_to_arcs.json'
with open(arcs_file_path, 'r') as file:
    arcs_data = json.load(file)

# Generate a network for each saga
for saga_name, saga_arcs in arc_segments.items():
    # Create a new graph for the saga
    G_saga = nx.Graph()

    # Iterate over the data to add nodes and relationships based on the saga arcs
    for person, details in data.items():
        word_count = details.get("word_count", 0)
        mutual_links = details.get("mention_intersection", [])
        character_arcs = arcs_data.get(person, [])

        # Check if the character belongs to any of the arcs in the current saga
        if any(arc in character_arcs for arc in saga_arcs):
            # Add the node to the graph with its 'word_count' attribute
            G_saga.add_node(person, word_count=word_count)

            # Add relationships with mutual mentions
            for mentioned in mutual_links:
                if mentioned in data:  # Ensure the mention exists in the data
                    G_saga.add_edge(person, mentioned)

    # Save the graph as a .gexf file
    relative_path = os.path.normpath(r'./../networks/one_piece_sagas')
    os.makedirs(relative_path, exist_ok=True)  # Ensure the directory exists
    filename = f"{saga_name.replace(' ', '_').lower()}.gexf"
    filepath = os.path.join(relative_path, filename)
    nx.write_gexf(G_saga, filepath)
    print(f"Archivo guardado: {filepath}")


Archivo guardado: ..\networks\one_piece_sagas\east_blue_saga.gexf
Archivo guardado: ..\networks\one_piece_sagas\alabasta_saga.gexf
Archivo guardado: ..\networks\one_piece_sagas\sky_island_saga.gexf
Archivo guardado: ..\networks\one_piece_sagas\water_7_saga.gexf
Archivo guardado: ..\networks\one_piece_sagas\thriller_bark_saga.gexf
Archivo guardado: ..\networks\one_piece_sagas\summit_war_saga.gexf
Archivo guardado: ..\networks\one_piece_sagas\fish-man_island_saga.gexf
Archivo guardado: ..\networks\one_piece_sagas\dressrosa_saga.gexf
Archivo guardado: ..\networks\one_piece_sagas\yonko_saga.gexf


# 